# Dec 16 – Problem 1

In [1]:
from collections import defaultdict
import heapq

with open("inputs/dec16.txt") as f:
    raw_data = f.read()

In [2]:
NORTH, EAST, SOUTH, WEST = 0, 1, 2, 3
deltas = [(0, -1), (1, 0), (0, 1), (-1, 0)]
width = len(raw_data.split("\n")[0])
pitch = width+1
height = len(raw_data.split("\n"))
map = [c for c in raw_data]

In [3]:
start_x, start_y = map.index("S") % pitch, map.index("S") // pitch
goal_x, goal_y = map.index("E") % pitch, map.index("E") // pitch
print(f"initial position {start_x, start_y}, goal {goal_x, goal_y}")
map[map.index("S")] = "."
map[map.index("E")] = "."

initial position (1, 139), goal (139, 1)


In [4]:
def get_cell(coords: tuple) -> str:
    x, y = coords
    if x < 0 or x >= width or y < 0 or y >= height:
        return None
    return map[y * pitch + x]

In [5]:
def path(c1: tuple, c2: tuple) -> tuple:
    x1, y1 = c1
    x2, y2 = c2
    return tuple(j for i in sorted([(x1, y1), (x2, y2)]) for j in i)

In [6]:
def dijkstra_search():
    distance = defaultdict(lambda:(float("inf"), set()))
    distance[(start_x, start_y, EAST)] = (0,set())
    queue = [(0, (start_x, start_y), EAST)]

    while queue:
        cost, coords, dir = heapq.heappop(queue)
        x, y = coords
        _, paths = distance[(x, y, dir)]
        for new_dir in range(-1, 2):
            dx, dy = deltas[(dir + new_dir) % 4]
            nx = x + dx
            ny = y + dy
            new_cost = cost + 1
            if new_dir != 0: new_cost += 1000

            while get_cell((nx, ny)) != "#" and get_cell((nx+dy, ny+dx)) == "#" and get_cell((nx-dy, ny-dx)) == "#":
                nx += dx
                ny += dy
                new_cost += 1
            
            new_paths = paths | {path((x, y), (nx, ny))}

            if nx == goal_x and ny == goal_y:
                return (new_cost, paths)
            
            if get_cell((nx, ny)) != "#":
                old_cost, old_paths = distance[(nx, ny, (dir + new_dir) % 4)]
                if old_cost == new_cost:
                    if any((seg not in old_paths) for seg in new_paths):
                        old_paths |= new_paths
                        heapq.heappush(queue, (new_cost, (nx, ny), (dir + new_dir) % 4))
                elif old_cost > new_cost:
                    distance[(nx, ny, (dir + new_dir) % 4)] = (new_cost, new_paths)
                    heapq.heappush(queue, (new_cost, (nx, ny), (dir + new_dir) % 4))

In [7]:
cost, _ = dijkstra_search()
print(cost)

92432


# Dec 16 – Problem 2

In [8]:
_, paths = dijkstra_search()
count = 0

for x1, y1, x2, y2 in paths:
    count += abs(x2 - x1) + abs(y2 - y1)

print(count)

458
